In [2]:
import pandas as pd
import mlflow
from preprocessing.preprocessing import *
preprocessing_version = 2

In [24]:
model_name = "lightgbm"
preprocessing_version = 2

# Load and test the model

From a terminal, run:
```
mlflow server --host 127.0.0.1 --port 5000
```

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
df = pd.read_csv("./data/consumption.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

print(f"Using preprocessing version {preprocessing_version}")
preprocessing = vars()[f"preprocessing_{preprocessing_version}"]
X, y = preprocessing(df)
print(f"X shape : {X.shape}")
print(f"y shape : {y.shape}")

Using preprocessing version 2


/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlforecast/core.py:455: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]
/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlforecast/core.py:455: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]


X shape : (944799, 99)
y shape : (944799,)


In [6]:
model_uri = 'runs:/4cfa339502cf4aafa1e30dd3c623fd6e/lgbm'
loaded_model = mlflow.lightgbm.load_model(model_uri)
loaded_model.predict(X[:1])

[LightGBM] [Warning] feature_fraction is set=0.592949529294766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.592949529294766


array([254.6388213])

# Deploy the model

In [66]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_name = f"Enefit {model_name} Preprocessing {preprocessing_version}"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id
print(f"Run ID : {experiment_id}")

Run ID : 847028692385838956


In [52]:
# Fetch the most recent run
runs = client.search_runs([experiment_id], order_by=["start_time DESC"], max_results=10)
# runs[0].to_dictionary()["data"]["metrics"]#["test_mae"]
if runs:
    latest_run_id = runs[0].info.run_id
    print(f"Latest Run ID: {latest_run_id}")

Latest Run ID: a991f56a0ea647c58b111bac380b7877


In [59]:
cmd = "mlflow models serve -m ./mlartifacts/847028692385838956/a991f56a0ea647c58b111bac380b7877/artifacts/lightgbm -p 5001".split(" ")

In [68]:
PORT = 1234
print(
    f"""Run the below command in a new window. You must be in the same repo as your mlruns directory and have mlflow installed :

    mlflow models serve -m ./mlartifacts/{experiment_id}/{latest_run_id}/artifacts/lightgbm -p {PORT}"""
)

Run the below command in a new window. You must be in the same repo as your mlruns directory and have mlflow installed :
    
    mlflow models serve -m ./mlartifacts/847028692385838956/a991f56a0ea647c58b111bac380b7877/artifacts/lightgbm -p 1234


# Call the deployed model

In [222]:
import requests
import json

# address = "127.0.0.1"
# address = "3.252.192.81"
address = "ec2-54-228-144-127.eu-west-1.compute.amazonaws.com"
port = 5000
endpoint = "api/predict"


In [223]:
url = f"http://{address}:{port}/"
r = requests.get(url)
print(r)
print(r.text)

<Response [200]>
Welcome to the homepage ! To get your energy forecasts, send your data to the /predict endpoint


In [224]:
with open("./data_json_mlforecast.json", "r") as f:
  data_json = json.load(f)

url = f"http://{address}:{port}/{endpoint}"
r = requests.post(url, headers={"Content-Type": "application/json"}, json=data_json)
r

<Response [200]>

In [225]:
print(r.json()[:1000])

{
    "predictions": [
        {
            "building_id": 0,
            "datetime": "2023-04-02 01:00:00",
            "forecasts": [
                922.1976036213131,
                932.5123610370592,
                931.3136118408446,
                931.8626423393976,
                933.6663574538877,
                904.9021517636521,
                848.4160493733832,
                718.1604980591101,
                585.8916252077718,
                471.4598847224826,
                363.03707176313384,
                321.75121793802043,
                315.6114474026614,
                312.5721677303937,
                339.64221853337955,
                416.8970230630121,
                549.0978898258145,
                699.4542516902336,
                845.9908312347918,
                964.3835680151163,
                1002.1501024082327,
                962.7871234020001,
                957.7852479536697,
                944.9739651173605
            ]
      